In this notebook,we will use Google Search results with the Gemini API in a technique called grounding, where the model is connected to verifiable sources of information. Here Gemini API automates a lot of it for i=us to search. The model generates Google Search queries and invokes the searches automatically, retrieving relevant data from Google's index of the web and providing links to search suggestions that support the query, so any user can verify the sources.Here we want to retrieve earthquake history and is available in google web and search the reasons of earthquake.

We need to Use the API.Please note we need use google api key(Which is hidden)
Start by installing and importing the Gemini API Python SDK

In [ ]:
!pip install -qU 'google-genai==1.7.0'

In [ ]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

To make the key available through colab -secrets, & add your key or enable it for this notebook.

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

The below command is sueful during an automated retry

In [ ]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

Search grounding is available in a limited set of models. Find a model that supports it on the models page.

In this note, you'll use gemini-2.0-flash for search grounding

Below shows how to make a request
To enable search grounding, you specify it as a tool: google_search. Like other tools, this is supplied as a parameter in GenerateContentConfig, and can be passed to generate_content calls as well as chats.create (for all chat turns) or chat.send_message (for specific turns).

In [ ]:
# Ask for information without search grounding.
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="Earthquake history of South East Asia from 2020-2025?")

Markdown(response.text)

The above is not giving info on specific earthquakes in Mynmar and thailand.This is the reason for enabling **grounding enabled**

In [ ]:
# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents="When did Mynmar and thailand Earthquake Occured?",
        config=config_with_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

**Below shows Response metadata**
When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

In [ ]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

Here we used standalone styled HTML content block that we use to link back to relevant search suggestions related to the generation.

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

The grounding_supports in the metadata provide a way for us to correlate the grounding chunks used to the generated output text.

In [ ]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

These supports can be used to highlight text in the response, or build tables of footnotes.

In [ ]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

**Search with tools**
In this below example, we are using the Google Search grounding tool and the code generation tool across two steps. In the first step, the model will use Google Search to find the requested information and then in the follow-up question, it generates code to plot the results.

This usage includes textual, visual and code parts, so first define a function to help visualise these.

In [ ]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())

        display(Markdown('----'))

Now we intiated a  chat asking for some information. Here we provide the Google Search tool so that the model can look up data from Google's Search index.

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="What was damage due to bankong earthquake in March 28th 2025?",
    config=config_with_search,
)

show_response(response)

Continuing the chat, now ask the model to convert the data into a chart. The code_execution tool is able to generate code to draw charts, execute that code and return the image. You can see the executed code in the executable_code part of the response.

Combining results from Google Search with tools like live plotting can enable very powerful use cases that require very little code to run.

In [ ]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Now plot this as a seaborn chart. Break out the earthquakes history in south east asia from 202-2025 too.",
    config=config_with_code,
)

show_response(response)

Now a Seaborn chart visualizing the earthquake history and occurance in south east asia in tabular form

In [ ]:
import requests
import pandas as pd

# Define the bounding box and query parameters
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "csv",
    "starttime": "2020-01-01",
    "endtime": "2025-04-12",
    "minlatitude": -11,
    "maxlatitude": 25,
    "minlongitude": 92,
    "maxlongitude": 142,
    "minmagnitude": 4.5,
    "orderby": "time"
}

# Fetch the data
response = requests.get(url, params=params)

# Save and read into pandas
with open("southeast_asia_earthquakes_2020_2025.csv", "w") as f:
    f.write(response.text)

df = pd.read_csv("southeast_asia_earthquakes_2020_2025.csv")
df.head()

Below libraries are used to plot a chart from 20-25 and shows frequency

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Optional: for nicer plots
sns.set(style="darkgrid")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x="mag", bins=30, kde=True)
plt.title("Distribution of Earthquake Magnitudes in Southeast Asia (2020–2025)")
plt.xlabel("Magnitude")
plt.ylabel("Frequency")
plt.show()

The below plot shows the magnitude of earthquakes occured over a period from 2020-2025

In [ ]:
df["time"] = pd.to_datetime(df["time"])  # make sure 'time' is a datetime object

plt.figure(figsize=(14, 6))
sns.scatterplot(data=df, x="time", y="mag", hue="mag", palette="viridis", size="mag", sizes=(20, 200))
plt.title("Earthquake Magnitudes Over Time in Southeast Asia (2020–2025)")
plt.xlabel("Date")
plt.ylabel("Magnitude")
plt.legend(title="Magnitude")
plt.show()

we used kdeplot to identify heat map generated in southeast asia and cleary seen red marked shows Mynmar/Thailand.

In [ ]:
plt.figure(figsize=(10, 8))
sns.kdeplot(
    data=df, x="longitude", y="latitude",
    cmap="Reds", fill=True, thresh=0.05
)
plt.title("Heatmap of Earthquake Locations in Southeast Asia")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

The below code visualized a epicenter depth and magnitude over a period of time

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="depth", y="mag", hue="mag", palette="coolwarm", alpha=0.7)
plt.title("Earthquake Depth vs. Magnitude (2020–2025)")
plt.xlabel("Depth (km)")
plt.ylabel("Magnitude")
plt.show()

The grounding techinque is used to get information on occurance of earthquakes in southeast asia and weburl and search tool helped to identified Mynmar and thailand Earthquaked.The above is used for Day 4 kaggle contest.